In [11]:
import numpy as np
import pandas as pd
import os
import sys
import zipfile
import subprocess
from datetime import datetime
from tqdm import tqdm
import json

# 基本配置
DATASET = 'ml-10m'
RAW_PATH = "D:/ML/recommendation/ReChorus/data/MovieLens_10M/"
TOPK_PATH = os.path.join(RAW_PATH, 'ML_10MTOPK/')
RANDOM_SEED = 0
NEG_ITEMS = 99

def get_time_range(hour):
    """获取时间段,与ML-1M保持一致"""
    if hour>=5 and hour<=8:
        return 0
    if hour>8 and hour<11:
        return 1
    if hour>=11 and hour<=12:
        return 2
    if hour>12 and hour<=15:
        return 3
    if hour>15 and hour<=17:
        return 4
    if hour>=18 and hour<=19:
        return 5
    if hour>19 and hour<=21:
        return 6
    if hour>21:
        return 7
    return 8 # 0-4 am

def download_and_extract():
    """下载并解压数据集"""
    if not os.path.exists(RAW_PATH):
        os.makedirs(RAW_PATH)
    if not os.path.exists(os.path.join(RAW_PATH, DATASET + '.zip')):
        print('Downloading data into ' + RAW_PATH)
        subprocess.call(
            'cd {} && curl -O http://files.grouplens.org/datasets/movielens/{}.zip'
            .format(RAW_PATH, DATASET), shell=True)
        print('Unzip files...')
        with zipfile.ZipFile(os.path.join(RAW_PATH, DATASET + '.zip'),'r') as f:
            f.extractall(RAW_PATH)

def process_data():
    """处理数据集"""
    print("Reading interaction data...")
    interactions = []
    user_freq, item_freq = dict(), dict()
    
    # 1. 读取评分数据
    with open(os.path.join(RAW_PATH, "ml-10M100K/ratings.dat")) as f:
        for line in tqdm(f):
            line = line.strip().split("::")
            uid, iid, rating, time = line[0], line[1], float(line[2]), float(line[3])
            if rating >= 4:  # 与ML-1M保持一致的正样本定义
                label = 1
            else:
                label = 0
            interactions.append([uid, time, iid, label])
            if int(label) == 1:
                user_freq[uid] = user_freq.get(uid, 0) + 1
                item_freq[iid] = item_freq.get(iid, 0) + 1

    # 2. 5-core过滤
    print("Performing 5-core filtering...")
    select_uid, select_iid = [], []
    while len(select_uid) < len(user_freq) or len(select_iid) < len(item_freq):
        select_uid, select_iid = [], []
        for u in user_freq:
            if user_freq[u] >= 5:
                select_uid.append(u)
        for i in item_freq:
            if item_freq[i] >= 5:
                select_iid.append(i)
        print(f"User: {len(select_uid)}/{len(user_freq)}, Item: {len(select_iid)}/{len(item_freq)}")

        select_uid = set(select_uid)
        select_iid = set(select_iid)
        user_freq, item_freq = dict(), dict()
        interactions_5core = []
        for line in interactions:
            uid, iid, label = line[0], line[2], line[-1]
            if uid in select_uid and iid in select_iid:
                interactions_5core.append(line)
                if int(label) == 1:
                    user_freq[uid] = user_freq.get(uid, 0) + 1
                    item_freq[iid] = item_freq.get(iid, 0) + 1
        interactions = interactions_5core

    # 3. 构建DataFrame并添加时间特征
    print("Processing timestamps...")
    interaction_df = pd.DataFrame(interactions, columns=["user_id", "time", "news_id", "label"])
    interaction_df['timestamp'] = interaction_df['time'].apply(lambda x: datetime.fromtimestamp(x))
    interaction_df['hour'] = interaction_df['timestamp'].apply(lambda x: x.hour)
    interaction_df['weekday'] = interaction_df['timestamp'].apply(lambda x: x.weekday())
    interaction_df['date'] = interaction_df['timestamp'].apply(lambda x: x.date())
    interaction_df['period'] = interaction_df.hour.apply(get_time_range)
    min_date = interaction_df.date.min()
    interaction_df['day'] = (interaction_df.date - min_date).apply(lambda x: x.days)

    # 4. 准备Top-k任务数据
    print("Preparing Top-k task data...")
    os.makedirs(TOPK_PATH, exist_ok=True)
    
    # 只保留正样本
    interaction_pos = interaction_df.loc[interaction_df.label==1].copy()
    interaction_pos.rename(columns={
        'hour': 'c_hour_c',
        'weekday': 'c_weekday_c',
        'period': 'c_period_c',
        'day': 'c_day_f',
        'user_id': 'original_user_id'
    }, inplace=True)

    # 重新映射ID
    user2newid = dict(zip(
        sorted(interaction_pos.original_user_id.unique()), 
        range(1, interaction_pos.original_user_id.nunique()+1)
    ))
    item2newid = dict(zip(
        sorted(interaction_pos.news_id.unique()), 
        range(1, interaction_pos.news_id.nunique()+1)
    ))
    
    interaction_pos['user_id'] = interaction_pos.original_user_id.apply(lambda x: user2newid[x])
    interaction_pos['item_id'] = interaction_pos.news_id.apply(lambda x: item2newid[x])

    # 5. 数据集划分
    split_time1 = int(interaction_pos.c_day_f.max() * 0.8)
    train = interaction_pos.loc[interaction_pos.c_day_f <= split_time1].copy()
    val_test = interaction_pos.loc[interaction_pos.c_day_f > split_time1].copy()
    val_test.sort_values(by='time', inplace=True)
    
    split_time2 = int(interaction_pos.c_day_f.max() * 0.9)
    val = val_test.loc[val_test.c_day_f <= split_time2].copy()
    test = val_test.loc[val_test.c_day_f > split_time2].copy()

    # 6. 处理元数据
    print("Processing metadata...")
    item_meta = pd.read_csv(
        os.path.join(RAW_PATH, "ml-10M100K/movies.dat"),
        sep='::', 
        names=['movieId', 'title', 'genres'],
        encoding='latin-1',
        engine='python'
    )
    
    item_select = item_meta.loc[item_meta.movieId.isin(interaction_pos.news_id.unique())].copy()
    item_select['item_id'] = item_select.movieId.apply(lambda x: item2newid[x])
    
    genres2id = dict(zip(
        sorted(item_select.genres.unique()),
        range(1, item_select.genres.nunique()+1)
    ))
    item_select['i_genre_c'] = item_select['genres'].apply(lambda x: genres2id[x])
    
    title2id = dict(zip(
        sorted(item_select.title.unique()),
        range(1, item_select.title.nunique()+1)
    ))
    item_select['i_title_c'] = item_select['title'].apply(lambda x: title2id[x])

    # 7. 保存数据
    print("Saving processed data...")
    columns = ['user_id', 'item_id', 'time', 'c_hour_c', 'c_weekday_c', 'c_period_c', 'c_day_f']
    
    # 保存训练集
    train[columns].to_csv(
        os.path.join(TOPK_PATH, 'train.csv'),
        sep='\t',
        index=False
    )
    
    # 为验证集和测试集生成负样本列
    for data, name in [(val, 'dev'), (test, 'test')]:
        data['neg_items'] = ''  # 这里可以添加负采样逻辑
        data[columns + ['neg_items']].to_csv(
            os.path.join(TOPK_PATH, f'{name}.csv'),
            sep='\t',
            index=False
        )
    
    # 保存元数据
    item_select[['item_id', 'i_genre_c', 'i_title_c']].to_csv(
        os.path.join(TOPK_PATH, 'item_meta.csv'),
        sep='\t',
        index=False
    )

if __name__ == "__main__":
    # download_and_extract()
    process_data()

Reading interaction data...


10000054it [00:31, 314202.95it/s]


Performing 5-core filtering...
User: 69167/69797, Item: 8790/10258
Processing timestamps...
Preparing Top-k task data...
Processing metadata...
Saving processed data...
